In [2]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
token = user_secrets.get_secret("github_repos_wildcard")

In [3]:
repo_url = f"https://{token}@github.com/gaserSami/panther.git"
branch = "autotuner"

In [4]:
!git clone -b {branch} {repo_url}

Cloning into 'panther'...
remote: Enumerating objects: 1657, done.
remote: Counting objects: 100% (329/329), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 1657 (delta 264), reused 277 (delta 241), pack-reused 1328 (from 1)
Receiving objects: 100% (1657/1657), 31.58 MiB | 18.14 MiB/s, done.
Resolving deltas: 100% (1099/1099), done.
Updating files: 100% (133/133), done.


In [5]:
# First uninstall existing torch, torchvision, torchaudio
!pip uninstall -y torch torchvision torchaudio

# Install the specified versions from PyTorch's official CUDA 12.4 wheels
!pip install torch==2.6.0+cu124 torchvision==0.21.0+cu124 torchaudio==2.6.0+cu124 --index-url https://download.pytorch.org/whl/cu124

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/

In [6]:
!mv panther Panther

In [7]:
%%writefile /kaggle/working/Panther/pawX/setup.py
from setuptools import setup
from torch.utils.cpp_extension import BuildExtension, CUDAExtension

setup(
    name="pawX",
    ext_modules=[
        CUDAExtension(
            name="pawX",
            sources=[
                "skops.cpp",
                "bindings.cpp",
                "linear.cpp",
                "linear_cuda.cu",
                "cqrrpt.cpp",
                "rsvd.cpp",
                "attention.cpp",
                "conv2d.cpp"
            ],
            # Use system includes and libraries
            include_dirs=["/usr/include/x86_64-linux-gnu"],
            library_dirs=[],
            libraries=["openblas"],
            extra_compile_args={"cxx": ["-O2", "-fopenmp"], "nvcc": ["-O2"]},
            extra_link_args=["-llapacke", "-lopenblas"]
        )
    ],
    cmdclass={"build_ext": BuildExtension},
)

Overwriting /kaggle/working/Panther/pawX/setup.py


In [8]:
!sudo apt-get install liblapacke-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  liblapacke libtmglib-dev libtmglib3
Suggested packages:
  liblapack-doc
The following NEW packages will be installed:
  liblapacke liblapacke-dev libtmglib-dev libtmglib3
0 upgraded, 4 newly installed, 0 to remove and 87 not upgraded.
Need to get 1,071 kB of archives.
After this operation, 12.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libtmglib3 amd64 3.10.0-2ubuntu1 [144 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblapacke amd64 3.10.0-2ubuntu1 [435 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libtmglib-dev amd64 3.10.0-2ubuntu1 [134 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblapacke-dev amd64 3.10.0-2ubuntu1 [358 kB]
Fetched 1,071 kB in 0s (7,645 kB/s)     
debconf: unable to initialize frontend: Dialog
debconf: (No usable dia

In [9]:
!cd /kaggle/working/Panther/pawX; python setup.py install
!cd /kaggle/working/Panther/pawX; pip install --no-build-isolation -e .

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
     

In [10]:
import torch

print(torch.__version__)
import triton

print(triton.__version__)

2.6.0+cu124
3.2.0


In [11]:
import os

os.chdir("/kaggle/working/Panther")

In [12]:
!pip install botorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.3/738.3 kB 16.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.7/277.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.3/176.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.3 MB/s eta 0:00:00


In [13]:
from panther.tuner.SkAutoTuner import *

# testing normal tree visualizer

In [14]:
import torch
import torch.nn as nn


# Create a simple test model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()

        # Some layers to test the visualization
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
        )

        self.classifier = nn.Sequential(
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


# Create model instance
model = SimpleModel()

# Test the fixed visualization
print("Model Tree with Types:")
ModelVisualizer.print_module_tree(model)

Model Tree with Types:
model (SimpleModel)/
└─ classifier (Sequential)/
│   ├─ 0 (Linear)
│   ├─ 1 (ReLU)
│   ├─ 2 (Dropout)
│   ├─ 3 (Linear)
└─ features (Sequential)/
    └─ 0 (Conv2d)
    └─ 1 (ReLU)
    └─ 2 (MaxPool2d)
    └─ 3 (Conv2d)
    └─ 4 (ReLU)


# testing interactive model visualizer on the simple model

In [15]:
def demonstrate_config_visualizer():
    """
    Demonstrates how to use the ConfigVisualizer to generate layer configurations
    for the SKAutoTuner.
    """
    # Create a simple model
    model = SimpleModel()

    print("Model created. Opening configuration visualizer...")
    print("Instructions:")
    print(
        "1. Ctrl+Click (or Cmd+Click on Mac) on layers in the visualization to select them"
    )
    print("2. Configure sketch parameters for the selected layers")
    print("3. Choose whether to tune layers separately or together")
    print("4. Generate and copy the configuration code")
    print("5. Use the generated code with SKAutoTuner")

    # Use the ConfigVisualizer to create an interactive visualization with configuration options
    output_path = ConfigVisualizer.create_config_visualization(
        model,
        output_path="model_config_visualization.html",  # Will be saved in the current directory
        open_browser=True,  # Set to True to automatically open in browser
    )

    print(f"\nConfiguration visualization saved to: {output_path}")
    print("If the browser doesn't open automatically, open the HTML file manually.")


if __name__ == "__main__":
    demonstrate_config_visualizer()

Model created. Opening configuration visualizer...
Instructions:
1. Ctrl+Click (or Cmd+Click on Mac) on layers in the visualization to select them
2. Configure sketch parameters for the selected layers
3. Choose whether to tune layers separately or together
4. Generate and copy the configuration code
5. Use the generated code with SKAutoTuner

Configuration visualization saved to: model_config_visualization.html
If the browser doesn't open automatically, open the HTML file manually.


# testing interactive model

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# Define a custom block with multiple layers
class CustomBlock(nn.Module):
    """A custom block with multiple layers"""

    def __init__(self, in_channels, out_channels, kernel_size=3):
        super(CustomBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels, out_channels, kernel_size, padding=kernel_size // 2
        )
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(
            out_channels, out_channels, kernel_size, padding=kernel_size // 2
        )
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        return x


# Define a custom model architecture that combines different types of layers
class CustomModel(nn.Module):
    """A custom model architecture that combines different types of layers"""

    def __init__(self, num_classes=10):
        super(CustomModel, self).__init__()

        # Encoder part with convolutional blocks
        self.encoder = nn.Sequential(
            CustomBlock(3, 64),
            nn.MaxPool2d(2, 2),
            CustomBlock(64, 128),
            nn.MaxPool2d(2, 2),
        )

        # Decoder for reconstruction with upsampling
        self.decoder = nn.Sequential(
            CustomBlock(128, 64),
            nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True),
            CustomBlock(64, 32),
            nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True),
            nn.Conv2d(32, 3, kernel_size=3, padding=1),
        )

        # Classifier with attention mechanism
        self.attention = nn.MultiheadAttention(
            embed_dim=128, num_heads=4, batch_first=True
        )

        # Final classifier
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, num_classes),
        )

    def forward(self, x):
        # Encode input
        features = self.encoder(x)

        # Reconstruct input using decoder
        reconstruction = self.decoder(features)

        # Apply self-attention to features
        batch_size, C, H, W = features.shape
        features_flat = features.view(batch_size, C, -1).permute(0, 2, 1)  # [B, HW, C]
        attn_output, _ = self.attention(features_flat, features_flat, features_flat)
        attn_output = attn_output.permute(0, 2, 1).view(batch_size, C, H, W)

        # Combine features and attention output
        features_combined = features + attn_output

        # Classify
        classification = self.classifier(features_combined)

        return classification, reconstruction


def main():
    # Create a model instance
    model = CustomModel(num_classes=10)

    print("Model created successfully.")
    print("\nModel structure (text representation):")
    # Print the model structure in text format
    ModelVisualizer.print_module_tree(model)

    print("\nCreating interactive visualization...")
    # Generate an interactive HTML visualization of the model
    output_path = ModelVisualizer.create_interactive_visualization(
        model,
        output_path="model_visualization.html",  # Will be saved in the current directory
        open_browser=True,  # Set to True to automatically open in browser
    )

    print(f"\nVisualization saved to: {output_path}")
    print(
        "Open the HTML file in a web browser to explore the interactive visualization."
    )
    print("Features of the visualization:")
    print("- Click on nodes to see detailed information about each layer")
    print("- Use the search bar to find specific layers")
    print("- Double-click on nodes to collapse/expand sections")
    print("- Mouse wheel or zoom buttons for zooming in/out")
    print("- Right-click on nodes for additional options")


if __name__ == "__main__":
    main()

Model created successfully.

Model structure (text representation):
model (CustomModel)/
└─ attention (MultiheadAttention)/
│   ├─ out_proj (NonDynamicallyQuantizableLinear)
└─ classifier (Sequential)/
│   ├─ 0 (AdaptiveAvgPool2d)
│   ├─ 1 (Flatten)
│   ├─ 2 (Linear)
│   ├─ 3 (ReLU)
│   ├─ 4 (Dropout)
│   ├─ 5 (Linear)
└─ decoder (Sequential)/
│   ├─ 0 (CustomBlock)/
│   │   ├─ bn1 (BatchNorm2d)
│   │   ├─ bn2 (BatchNorm2d)
│   │   ├─ conv1 (Conv2d)
│   │   ├─ conv2 (Conv2d)
│   ├─ 1 (Upsample)
│   ├─ 2 (CustomBlock)/
│   │   ├─ bn1 (BatchNorm2d)
│   │   ├─ bn2 (BatchNorm2d)
│   │   ├─ conv1 (Conv2d)
│   │   ├─ conv2 (Conv2d)
│   ├─ 3 (Upsample)
│   ├─ 4 (Conv2d)
└─ encoder (Sequential)/
    └─ 0 (CustomBlock)/
    │   ├─ bn1 (BatchNorm2d)
    │   ├─ bn2 (BatchNorm2d)
    │   ├─ conv1 (Conv2d)
    │   ├─ conv2 (Conv2d)
    └─ 1 (MaxPool2d)
    └─ 2 (CustomBlock)/
    │   ├─ bn1 (BatchNorm2d)
    │   ├─ bn2 (BatchNorm2d)
    │   ├─ conv1 (Conv2d)
    │   ├─ conv2 (Conv2d)
    └─ 3 (MaxP

# more complex interactive visualizer testing

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(
            in_channels, out_channels, kernel_size, padding=kernel_size // 2
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x


class SimpleConvNet(nn.Module):
    """
    A simple convolutional network for demonstrating the ConfigVisualizer
    with Conv2d layers.
    """

    def __init__(self, in_channels=3, num_classes=10):
        super(SimpleConvNet, self).__init__()

        # Encoder blocks
        self.encoder = nn.Sequential(
            ConvBlock(in_channels, 32),
            nn.MaxPool2d(2, 2),
            ConvBlock(32, 64),
            nn.MaxPool2d(2, 2),
            ConvBlock(64, 128),
            nn.MaxPool2d(2, 2),
        )

        # Linear layers
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, num_classes),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.classifier(x)
        return x


class TransformerModel(nn.Module):
    """
    A simple transformer-based model for demonstrating the ConfigVisualizer
    with attention layers.
    """

    def __init__(
        self,
        vocab_size=5000,
        d_model=256,
        nhead=8,
        num_layers=2,
        dim_feedforward=1024,
        max_seq_length=100,
        num_classes=10,
    ):
        super(TransformerModel, self).__init__()

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = nn.Parameter(torch.zeros(max_seq_length, d_model))

        # Transformer encoder
        encoder_layers = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layers, num_layers=num_layers
        )

        # Self-attention layer
        self.attention = nn.MultiheadAttention(d_model, nhead)

        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Linear(d_model // 2, num_classes),
        )

    def forward(self, x):
        # Add position encoding
        seq_length = x.size(1)
        x = self.embedding(x)
        x = x + self.pos_encoder[:seq_length, :]

        # Reshape for transformer: [seq_len, batch, dim]
        x = x.permute(1, 0, 2)

        # Apply transformer
        x = self.transformer_encoder(x)

        # Self-attention on the output sequence
        attn_output, _ = self.attention(x, x, x)

        # Average pooling across sequence dimension
        x = attn_output.mean(dim=0)

        # Classification
        x = self.classifier(x)
        return x


def demonstrate_complex_models():
    """
    Demonstrates the ConfigVisualizer with both convolutional and transformer models
    """
    # Ask which model to visualize
    print("Which model would you like to visualize?")
    print("1. Convolutional Neural Network")
    print("2. Transformer Model")

    choice = input("Enter your choice (1 or 2): ")

    if choice == "1":
        # Create a CNN model
        model = SimpleConvNet()
        model_name = "cnn"
    else:
        # Create a Transformer model
        model = TransformerModel()
        model_name = "transformer"

    print(
        f"\nModel created. Opening configuration visualizer for {model_name} model..."
    )
    print("Instructions:")
    print(
        "1. Ctrl+Click (or Cmd+Click on Mac) on layers in the visualization to select them"
    )
    print("2. Configure sketch parameters for the selected layers")
    print("3. Choose whether to tune layers separately or together")
    print("4. Generate and copy the configuration code")
    print("5. Use the generated code with SKAutoTuner")

    # Use the ConfigVisualizer
    output_path = ConfigVisualizer.create_config_visualization(
        model, output_path=f"{model_name}_config_visualization.html", open_browser=True
    )

    print(f"\nConfiguration visualization saved to: {output_path}")


if __name__ == "__main__":
    demonstrate_complex_models()

Which model would you like to visualize?
1. Convolutional Neural Network
2. Transformer Model


Enter your choice (1 or 2):  1



Model created. Opening configuration visualizer for cnn model...
Instructions:
1. Ctrl+Click (or Cmd+Click on Mac) on layers in the visualization to select them
2. Configure sketch parameters for the selected layers
3. Choose whether to tune layers separately or together
4. Generate and copy the configuration code
5. Use the generated code with SKAutoTuner

Configuration visualization saved to: cnn_config_visualization.html
